In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# パラメタ
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# ネットワークパラメタ
n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = {
    "out": tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    "out": tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
def RNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_steps, 0)

    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter:", str(step*batch_size), "Loss=", loss, "Acc=", acc
        step += 1
    print "Optimization Finish"
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Acc:", sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter: 1280 Loss= 1.87096 Acc= 0.359375
Iter: 2560 Loss= 1.63714 Acc= 0.476562
Iter: 3840 Loss= 1.27163 Acc= 0.546875
Iter: 5120 Loss= 1.07733 Acc= 0.65625
Iter: 6400 Loss= 0.852438 Acc= 0.679688
Iter: 7680 Loss= 1.10693 Acc= 0.617188
Iter: 8960 Loss= 0.842882 Acc= 0.71875
Iter: 10240 Loss= 0.604242 Acc= 0.8125
Iter: 11520 Loss= 0.405936 Acc= 0.914062
Iter: 12800 Loss= 0.71085 Acc= 0.765625
Iter: 14080 Loss= 0.476718 Acc= 0.84375
Iter: 15360 Loss= 0.340632 Acc= 0.90625
Iter: 16640 Loss= 0.440785 Acc= 0.90625
Iter: 17920 Loss= 0.354283 Acc= 0.882812
Iter: 19200 Loss= 0.28498 Acc= 0.898438
Iter: 20480 Loss= 0.169025 Acc= 0.953125
Iter: 21760 Loss= 0.408473 Acc= 0.851562
Iter: 23040 Loss= 0.190999 Acc= 0.929688
Iter: 24320 Loss= 0.406992 Acc= 0.867188
Iter: 25600 Loss= 0.402364 Acc= 0.882812
Iter: 26880 Loss= 0.253933 Acc= 0.890625
Iter: 28160 Loss= 0.279674 Acc= 0.898438
Iter: 29440 Loss= 0.309937 Acc= 0.882812
Iter: 30720 Loss= 0.323382 Acc= 0.875
Iter: 32000 Loss= 0.221271 Acc= 0.929688